In [1]:
import numpy as np
import pandas as p

In [2]:
data = p.read_csv('train.csv', sep=',')

In [3]:
data.head()

,line num,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [4]:
data.shape

(103904, 25)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 25 columns):
line num                             103904 non-null int64
id                                   103904 non-null int64
Gender                               103904 non-null object
Customer Type                        103904 non-null object
Age                                  103904 non-null int64
Type of Travel                       103904 non-null object
Class                                103904 non-null object
Flight Distance                      103904 non-null int64
Inflight wifi service                103904 non-null int64
Departure/Arrival time convenient    103904 non-null int64
Ease of Online booking               103904 non-null int64
Gate location                        103904 non-null int64
Food and drink                       103904 non-null int64
Online boarding                      103904 non-null int64
Seat comfort                         103904 non-null int64


Перетираем поля Arrival Delay со значениями null значением из поля Departure Delay

In [6]:
nullrows = data[data['Arrival Delay in Minutes'].isnull()].index
for idx in nullrows:
    data.loc[idx, 'Arrival Delay in Minutes'] = data.loc[idx, 'Departure Delay in Minutes']

In [7]:
data = data.drop(['line num', 'id'], axis=1)

In [8]:
targets = data.groupby('satisfaction').size()/len(data)
print(targets)

satisfaction
neutral or dissatisfied    0.566667
satisfied                  0.433333
dtype: float64


In [9]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [10]:
y = le.fit_transform(data['satisfaction'])
print(list(le.classes_))
print(y)

['neutral or dissatisfied', 'satisfied']
[0 0 1 ... 0 0 0]


In [11]:
data = data.drop(['satisfaction'], axis=1)

In [12]:
X2 = data.select_dtypes(include=[object])
X2.head(3)

,Gender,Customer Type,Type of Travel,Class
0,Male,Loyal Customer,Personal Travel,Eco Plus
1,Male,disloyal Customer,Business travel,Business
2,Female,Loyal Customer,Business travel,Business


In [13]:
columnsToEncode = data.select_dtypes(include=[object]).columns.values

In [37]:
mlb = preprocessing.OrdinalEncoder()
dd0 = mlb.fit_transform(data[['Gender','Customer Type','Type of Travel']])
print(dd0)

[[1. 0. 1.]
 [1. 1. 0.]
 [0. 0. 0.]
 ...
 [1. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [38]:
data[['Gender','Customer Type','Type of Travel']] = p.DataFrame(dd0)

In [39]:
data.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,Business,Eco,Eco Plus
0,1.0,0.0,13,1.0,Eco Plus,460,3,4,3,1,...,3,4,4,5,5,25,18.0,0.0,0.0,1.0
1,1.0,1.0,25,0.0,Business,235,3,2,3,3,...,5,3,1,4,1,1,6.0,1.0,0.0,0.0
2,0.0,0.0,26,0.0,Business,1142,2,2,2,2,...,3,4,4,4,5,0,0.0,1.0,0.0,0.0
3,0.0,0.0,25,0.0,Business,562,2,5,5,5,...,5,3,1,4,2,11,9.0,1.0,0.0,0.0
4,1.0,0.0,61,0.0,Business,214,3,3,3,3,...,4,4,3,3,3,0,0.0,1.0,0.0,0.0


In [40]:
ohe = preprocessing.OneHotEncoder()
dd = ohe.fit_transform(data['Class'].values.reshape(-1, 1)).toarray()
feature_labels = ohe.categories_
feature_labels = np.concatenate(feature_labels)
print(feature_labels)

['Business' 'Eco' 'Eco Plus']


In [41]:
data[feature_labels] = p.DataFrame(dd)

In [42]:
data.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,Business,Eco,Eco Plus
0,1.0,0.0,13,1.0,Eco Plus,460,3,4,3,1,...,3,4,4,5,5,25,18.0,0.0,0.0,1.0
1,1.0,1.0,25,0.0,Business,235,3,2,3,3,...,5,3,1,4,1,1,6.0,1.0,0.0,0.0
2,0.0,0.0,26,0.0,Business,1142,2,2,2,2,...,3,4,4,4,5,0,0.0,1.0,0.0,0.0
3,0.0,0.0,25,0.0,Business,562,2,5,5,5,...,5,3,1,4,2,11,9.0,1.0,0.0,0.0
4,1.0,0.0,61,0.0,Business,214,3,3,3,3,...,4,4,3,3,3,0,0.0,1.0,0.0,0.0


In [43]:
data = data.drop(['Class'], axis=1)

In [44]:
data.head()

,Gender,Customer Type,Age,Type of Travel,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,...,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,Business,Eco,Eco Plus
0,1.0,0.0,13,1.0,460,3,4,3,1,5,...,3,4,4,5,5,25,18.0,0.0,0.0,1.0
1,1.0,1.0,25,0.0,235,3,2,3,3,1,...,5,3,1,4,1,1,6.0,1.0,0.0,0.0
2,0.0,0.0,26,0.0,1142,2,2,2,2,5,...,3,4,4,4,5,0,0.0,1.0,0.0,0.0
3,0.0,0.0,25,0.0,562,2,5,5,5,2,...,5,3,1,4,2,11,9.0,1.0,0.0,0.0
4,1.0,0.0,61,0.0,214,3,3,3,3,4,...,4,4,3,3,3,0,0.0,1.0,0.0,0.0


In [45]:
data.corr()

,Gender,Customer Type,Age,Type of Travel,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,...,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,Business,Eco,Eco Plus
Gender,1.000000,-0.031732,0.008928,0.006909,0.005828,0.008974,0.009142,0.006951,0.000333,0.005772,...,0.031798,0.037391,0.010563,0.039410,0.006505,0.002935,0.000802,0.010634,-0.005295,-0.010351
Customer Type,-0.031732,1.000000,-0.281674,-0.308333,-0.225200,-0.007543,-0.206869,-0.019509,0.006078,-0.059454,...,-0.047868,0.024723,-0.032177,0.022670,-0.083613,0.004036,0.004459,-0.085601,0.118694,-0.062967
Age,0.008928,-0.281674,1.000000,-0.048524,0.099461,0.017859,0.038125,0.024842,-0.001330,0.023000,...,0.040583,-0.047529,0.035482,-0.049427,0.053611,-0.010152,-0.012137,0.138939,-0.132551,-0.013376
Type of Travel,0.006909,-0.308333,-0.048524,1.000000,-0.267344,-0.105085,0.259763,-0.133511,-0.030700,-0.063439,...,-0.138803,-0.031321,0.016751,-0.022599,-0.079207,-0.005432,-0.005086,-0.552125,0.500938,0.102775
Flight Distance,0.005828,-0.225200,0.099461,-0.267344,1.000000,0.007131,-0.020043,0.065717,0.004793,0.056994,...,0.133916,0.063184,0.073072,0.057540,0.093149,0.002158,-0.002154,0.466896,-0.404493,-0.123674
Inflight wifi service,0.008974,-0.007543,0.017859,-0.105085,0.007131,1.000000,0.343845,0.715856,0.336248,0.134718,...,0.160473,0.120923,0.043193,0.110441,0.132698,-0.017402,-0.019038,0.032885,-0.037197,0.008034
Departure/Arrival time convenient,0.009142,-0.206869,0.038125,0.259763,-0.020043,0.343845,1.000000,0.436961,0.444757,0.004906,...,0.012441,0.072126,0.093333,0.073318,0.014292,0.001005,-0.000645,-0.096870,0.082321,0.028740
Ease of Online booking,0.006951,-0.019509,0.024842,-0.133511,0.065717,0.715856,0.436961,1.000000,0.458655,0.031873,...,0.107601,0.038762,0.011081,0.035272,0.016179,-0.006371,-0.008051,0.107436,-0.098039,-0.018914
Gate location,0.000333,0.006078,-0.001330,-0.030700,0.004793,0.336248,0.444757,0.458655,1.000000,-0.001159,...,-0.005873,0.002313,-0.035427,0.001681,-0.003830,0.005467,0.005082,0.004526,-0.003488,-0.002031
Food and drink,0.005772,-0.059454,0.023000,-0.063439,0.056994,0.134718,0.004906,0.031873,-0.001159,1.000000,...,0.032498,0.034746,0.087299,0.033993,0.657760,-0.029926,-0.032455,0.087114,-0.078801,-0.016671
